In [ ]:
import pandas as pd


In [ ]:
pip install langdetect

In [34]:
pip install googletrans==4.0.0-rc1


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 3.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.2 MB/s eta 0:00:0000:0100:01
  Using cached rfc3986-1.5.0-py2.py3-none-any.whl (31 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 8.3 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17416 sha256=2dd92068269b616f04f872e770aa6468ec9029181a6881d110c5f95e5f6bd56d
  Stored in directory: /home/est/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [6]:
#COD 1
#completando o cod
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd

options = webdriver.FirefoxOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors=yes')
options.add_argument('--start-maximized')

# Inicializa o navegador
firefox = webdriver.Remote(command_executor="http://localhost:4444/wd/hub", options=options)

# Lista para armazenar todas as avaliações
all_reviews = []

# Número total de páginas de avaliações (você pode ajustar esse valor conforme necessário)
total_pages = 5

# URLs das páginas de avaliações
base_urls = [
    "https://www.amazon.com.br/Unique-Stories-conte%C3%BAdo-fortalecer-oportunidades/product-reviews/8582892136/ref=cm_cr_arp_d_paging_btm_prev_1?ie=UTF8&pageNumber={}&reviewerType=all_reviews",
    "https://www.amazon.com.br/Write-KILLER-LinkedIn-Profile-Mistakes-ebook/product-reviews/B019ELXR8W/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
]

for base_url in base_urls:
    # Lista para armazenar todas as avaliações do link atual
    link_reviews = []

    for page_number in range(1, total_pages + 1):
        url = base_url.format(page_number)

        # Abre a URL
        firefox.get(url)

        # Espera um tempo para que a página carregue completamente
        sleep(10)

        # Extrai as informações das avaliações
        reviews = []

        # Encontra todos os elementos que contêm informações das avaliações
        review_elements = firefox.find_elements(By.CSS_SELECTOR, "div.a-section.review.aok-relative")

        for review_element in review_elements:
            # Extrai as informações de cada avaliação
            author = review_element.find_element(By.CSS_SELECTOR, "span.a-profile-name").text
            rating = review_element.find_element(By.CSS_SELECTOR, "span.a-icon-alt").get_attribute("innerHTML")
            
            # Converte o rating para numérico mantendo o padrão brasileiro
            rating = float(rating.split(" ")[0].replace(",", "."))


            # Título
            try:
                title_element = review_element.find_element(By.CSS_SELECTOR, "a.review-title span")
                title = title_element.text if title_element else "Sem título"
            except:
                title = "Sem título"

            content = review_element.find_element(By.CSS_SELECTOR, "span.review-text-content span").text
            date = review_element.find_element(By.CSS_SELECTOR, "span.review-date").text

            # Verifica se a avaliação é verificada
            verified_element = review_element.find_elements(By.XPATH, ".//span[contains(text(), 'Compra verificada')]")
            verified = "Compra verificada" if verified_element else "Não verificada"

            # Tenta encontrar a URL da imagem da avaliação
            try:
                image_url = review_element.find_element(By.CSS_SELECTOR, "img.review-image-tile").get_attribute("src")
            except:
                image_url = None

            # Título
            try:
                title_element = review_element.find_element(By.XPATH, ".//a[@data-hook='review-title']")
                title = title_element.text if title_element else "Sem título"
            except:
                title = "Sem título"

            try:
                useful_info = review_element.find_element(By.XPATH, ".//span[contains(text(), 'pessoa achou isso útil')]").text
            except:
                useful_info = "Informação útil não encontrada"

            # Adiciona as informações à lista de avaliações
            reviews.append({
                "URL": url,
                "author": author,
                "rating": rating,
                "title": title,
                "content": content,
                "date": date,
                "verified": verified,
                "image_url": image_url,
                "useful_info": useful_info
            })

        # Adiciona as avaliações desta página à lista geral de avaliações
        all_reviews.extend(reviews)

# Fechar o navegador
firefox.quit()

# Cria um DataFrame a partir da lista de avaliações
df = pd.DataFrame(all_reviews)

# Exibe o DataFrame
df



,URL,author,rating,title,content,date,verified,image_url,useful_info
0,https://www.amazon.com.br/Unique-Stories-conte...,Cláudio Soares do Nascimento,5.0,Ótimo livro para você aprender a valorizar a s...,Esse livro não ensina só sobre LinkedIn e posi...,Avaliado no Brasil em 22 de setembro de 2022,Compra verificada,None,1 pessoa achou isso útil
1,https://www.amazon.com.br/Unique-Stories-conte...,Lucélie Zancanaro - Estrategista de Marketing ...,5.0,Unique Stories,Fazia algum tempo que eu a acompanhava no Link...,Avaliado no Brasil em 17 de dezembro de 2020,Compra verificada,https://images-na.ssl-images-amazon.com/images...,Informação útil não encontrada
2,https://www.amazon.com.br/Unique-Stories-conte...,Sarah Oliveira,4.0,Obras adquiridas !,"EU ainda não li a obra, pois estou terminando ...",Avaliado no Brasil em 19 de maio de 2023,Compra verificada,https://m.media-amazon.com/images/I/51hkmVGTql...,Informação útil não encontrada
3,https://www.amazon.com.br/Unique-Stories-conte...,Natália Soares,5.0,Livro maravilhoso,Já conheço o trabalho da Flávia e gosto muito ...,Avaliado no Brasil em 2 de outubro de 2023,Compra verificada,None,Informação útil não encontrada
4,https://www.amazon.com.br/Unique-Stories-conte...,CLAYTON SILVEIRA LEITE,3.0,Poderia baixar o livro em pdf,"O livro poderia também ser baixado em pdf, me ...",Avaliado no Brasil em 19 de julho de 2019,Compra verificada,None,1 pessoa achou isso útil
...,...,...,...,...,...,...,...,...,...
60,https://www.amazon.com.br/Write-KILLER-LinkedI...,Edgar Bermudez Serrano,5.0,Sem título,Theres a lot of good tips to learn about how t...,Avaliado no México em 12 de julho de 2020,Compra verificada,None,Informação útil não encontrada
61,https://www.amazon.com.br/Write-KILLER-LinkedI...,Pinoko_NY,5.0,Sem título,Since the COVID-19 the world has changed. we h...,Avaliado no Japão em 14 de outubro de 2020,Compra verificada,None,Informação útil não encontrada
62,https://www.amazon.com.br/Write-KILLER-LinkedI...,Stefano Parravicini,5.0,Sem título,When I started reading this book I knew almost...,Avaliado na Itália em 2 de junho de 2018,Compra verificada,None,Informação útil não encontrada
63,https://www.amazon.com.br/Write-KILLER-LinkedI...,JP Cofre,4.0,Sem título,It reads easily and the material is quite down...,Avaliado nos Estados Unidos em 22 de julho de ...,Compra verificada,None,Informação útil não encontrada


In [36]:
#COD 2
# usar para tradução em portugues caso venha sem
from googletrans import Translator

from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

options = webdriver.FirefoxOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors=yes')
options.add_argument('--start-maximized')

# Inicializa o navegador
firefox = webdriver.Remote(command_executor="http://localhost:4444/wd/hub", options=options)

# Lista para armazenar todas as avaliações
all_reviews = []

# Número total de páginas de avaliações (você pode ajustar esse valor conforme necessário)
total_pages = 5

# URLs das páginas de avaliações
base_urls = [
    #"https://www.amazon.com.br/Unique-Stories-conte%C3%BAdo-fortalecer-oportunidades/product-reviews/8582892136/ref=cm_cr_arp_d_paging_btm_prev_1?ie=UTF8&pageNumber={}&reviewerType=all_reviews",
    "https://www.amazon.com.br/Write-KILLER-LinkedIn-Profile-Mistakes-ebook/product-reviews/B019ELXR8W/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
]

for base_url in base_urls:
    # Lista para armazenar todas as avaliações do link atual
    link_reviews = []

    for page_number in range(1, total_pages + 1):
        url = base_url.format(page_number)

        # Abre a URL
        firefox.get(url)

        # Espera um tempo para que a página carregue completamente
        sleep(10)

        # Extrai as informações das avaliações
        reviews = []

        # Encontra todos os elementos que contêm informações das avaliações
        review_elements = firefox.find_elements(By.CSS_SELECTOR, "div.a-section.review.aok-relative")

        for review_element in review_elements:
            # Extrai as informações de cada avaliação
            author = review_element.find_element(By.CSS_SELECTOR, "span.a-profile-name").text
            rating = review_element.find_element(By.CSS_SELECTOR, "span.a-icon-alt").get_attribute("innerHTML")
            
            # Converte o rating para numérico mantendo o padrão brasileiro
            rating = float(rating.split(" ")[0].replace(",", "."))

            # Título
            try:
                title_element = review_element.find_element(By.CSS_SELECTOR, "a.review-title span")
                title = title_element.text if title_element else "Sem título"
            except:
                title = "Sem título"

            content = review_element.find_element(By.CSS_SELECTOR, "span.review-text-content span").text
            date = review_element.find_element(By.CSS_SELECTOR, "span.review-date").text

            # Verifica se a avaliação é verificada
            verified_element = review_element.find_elements(By.XPATH, ".//span[contains(text(), 'Compra verificada')]")
            verified = "Compra verificada" if verified_element else "Não verificada"

            # Tenta encontrar a URL da imagem da avaliação
            try:
                image_url = review_element.find_element(By.CSS_SELECTOR, "img.review-image-tile").get_attribute("src")
            except:
                image_url = None

            # Título
            try:
                title_element = review_element.find_element(By.XPATH, ".//a[@data-hook='review-title']")
                title = title_element.text if title_element else "Sem título"
            except:
                title = "Sem título"

            try:
                useful_info = review_element.find_element(By.XPATH, ".//span[contains(text(), 'pessoa achou isso útil')]").text
            except:
                useful_info = "Informação útil não encontrada"

            # Detecta o idioma do conteúdo e converte para o português, se necessário
            try:
                translator = Translator()
                if detect(content) != 'pt':
                    content = content.translate(str.maketrans('', '', '\n\t'))
                    content = translator.translate(content, src='auto', dest='pt').text
            except LangDetectException:
                pass  # Se a detecção de idioma falhar, mantenha o conteúdo original

            # Adiciona as informações à lista de avaliações
            reviews.append({
                "URL": url,
                "author": author,
                "rating": rating,
                "title": title,
                "content": content,
                "date": date,
                "verified": verified,
                "image_url": image_url,
                "useful_info": useful_info
            })

        # Adiciona as avaliações desta página à lista geral de avaliações
        all_reviews.extend(reviews)

# Fechar o navegador
firefox.quit()

# Cria um DataFrame a partir da lista de avaliações
df = pd.DataFrame(all_reviews)

# Exibe o DataFrame
df


,URL,author,rating,title,content,date,verified,image_url,useful_info
0,https://www.amazon.com.br/Write-KILLER-LinkedI...,WILLIAM MAZZA,5.0,The best Linkedin book ever,"É um livro muito útil, com toneladas de dicas ...",Avaliado no Brasil em 27 de dezembro de 2018,Compra verificada,None,Informação útil não encontrada
1,https://www.amazon.com.br/Write-KILLER-LinkedI...,Paul C,5.0,Sem título,Melhor da aula - o benchmark para profissionais,Avaliado no Reino Unido em 27 de julho de 2023,Compra verificada,None,Informação útil não encontrada
2,https://www.amazon.com.br/Write-KILLER-LinkedI...,MG,5.0,Sem título,Ótimo manual para tirar o melhor proveito do p...,Avaliado na França em 19 de novembro de 2020,Compra verificada,None,Informação útil não encontrada
3,https://www.amazon.com.br/Write-KILLER-LinkedI...,Andrew Hallahan,5.0,Sem título,"Depois de ouvir Brenda em um podcast, fiquei i...",Avaliado na Austrália em 7 de julho de 2019,Compra verificada,None,Informação útil não encontrada
4,https://www.amazon.com.br/Write-KILLER-LinkedI...,Nitan Shanas,5.0,Sem título,Este livro valeu a pena os US $ 10.Agora tenho...,Avaliado nos Estados Unidos em 25 de abril de ...,Compra verificada,None,Informação útil não encontrada
5,https://www.amazon.com.br/Write-KILLER-LinkedI...,Edgar Bermudez Serrano,5.0,Sem título,Há muitas boas dicas para aprender sobre como ...,Avaliado no México em 12 de julho de 2020,Compra verificada,None,Informação útil não encontrada
6,https://www.amazon.com.br/Write-KILLER-LinkedI...,Pinoko_NY,5.0,Sem título,"Desde que o covid-19, o mundo mudou.Temos que ...",Avaliado no Japão em 14 de outubro de 2020,Compra verificada,None,Informação útil não encontrada
7,https://www.amazon.com.br/Write-KILLER-LinkedI...,Stefano Parravicini,5.0,Sem título,"Quando comecei a ler este livro, não sabia qua...",Avaliado na Itália em 2 de junho de 2018,Compra verificada,None,Informação útil não encontrada
8,https://www.amazon.com.br/Write-KILLER-LinkedI...,JP Cofre,4.0,Sem título,Ele lê facilmente e o material é bastante pé n...,Avaliado nos Estados Unidos em 22 de julho de ...,Compra verificada,None,Informação útil não encontrada
9,https://www.amazon.com.br/Write-KILLER-LinkedI...,Nata,5.0,Sem título,Eu só tenho que dizer que funciona.E é realmen...,Avaliado na Itália em 21 de agosto de 2018,Compra verificada,None,Informação útil não encontrada
